In [2]:
import mysql.connector
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.float_format', '{:.2f}'.format)


In [4]:
df=pd.read_csv('data.csv')

In [8]:

# 1. Calcular elasticidades por productLine con regresión log-log
models_by_productline = {}
elasticities_by_productline = {}

for product in df['productLine'].unique():
    df_product = df[df['productLine'] == product]
    df_product['log_price'] = np.log(df_product['priceEach'])
    df_product['log_quantity'] = np.log(df_product['quantityOrdered'])
    
    # Regresión log-log
    X = sm.add_constant(df_product['log_price'])
    y = df_product['log_quantity']
    model = sm.OLS(y, X).fit()
    
    models_by_productline[product] = model
    elasticities_by_productline[product] = model.params['log_price']  # Pendiente como elasticidad

# 2. Calcular precios óptimos con lógica ajustada según elasticidad
current_prices = df.groupby('productLine')['priceEach'].mean()
optimal_prices = {}

for product, elasticity in elasticities_by_productline.items():
    if elasticity > 0:  # Elasticidad positiva (productos de lujo)
        # Subir el precio para productos de lujo
        optimal_price = current_prices[product] * 1.1
    elif elasticity < -1:  # Elasticidad mayor que -1 (demanda elástica)
        # Bajar el precio para productos con demanda elástica
        optimal_price = current_prices[product] / (1 + abs(1 / elasticity))
    elif elasticity >= -1 and elasticity < 0:  # Elasticidad entre -1 y 0 (demanda inelástica)
        # Subir el precio para productos con demanda inelástica
        optimal_price = current_prices[product] * (1 + abs(1 / elasticity))
    else:  # Elasticidad cero (sin cambio)
        optimal_price = current_prices[product]

    # Restricción de precio positivo
    if optimal_price > 0 and not np.isinf(optimal_price):  # Verificar que no sea infinito
        optimal_prices[product] = optimal_price
    else:
        optimal_prices[product] = current_prices[product]  # Fallback en casos extremos

# Agregar información adicional en la comparación
comparison = pd.DataFrame({
    "Precio Actual": current_prices,
    "Precio Óptimo": optimal_prices,
    "Elasticidad": elasticities_by_productline,
})

# Clasificación de elasticidad
def classify_elasticity(elasticity):
    if elasticity > 0:
        return "Elástica positiva (productos de lujo)"
    elif elasticity < -1:
        return "Elástica (Demanda elástica)"
    elif -1 <= elasticity < 0:
        return "Inelástica (Demanda inelástica)"
    else:
        return "Sin cambio"

comparison['Tipo de Demanda'] = comparison['Elasticidad'].apply(classify_elasticity)

# Calcular la variación porcentual del precio
comparison['Variación Porcentual'] = ((comparison['Precio Óptimo'] - comparison['Precio Actual']) / comparison['Precio Actual']) * 100

# Calcular el ingreso antes y después del cambio de precios
df['ingreso_actual'] = df['priceEach'] * df['quantityOrdered']
df['ingreso_optimo'] = df['priceEach'] * df['quantityOrdered']  # Inicializamos con precio actual

# Calcular el ingreso óptimo para cada producto (basado en el precio óptimo)
for product, optimal_price in optimal_prices.items():
    df_product = df[df['productLine'] == product]
    df_product['ingreso_optimo'] = optimal_price * df_product['quantityOrdered']
    df.loc[df_product.index, 'ingreso_optimo'] = df_product['ingreso_optimo']

# Calcular el ingreso promedio antes y después del cambio
ingreso_actual_promedio = df.groupby('productLine')['ingreso_actual'].mean()
ingreso_optimo_promedio = df.groupby('productLine')['ingreso_optimo'].mean()

# Calcular el ingreso total general antes y después del cambio de precios
ingreso_total_actual = df['ingreso_actual'].sum()
ingreso_total_optimo = df['ingreso_optimo'].sum()

# Agregar ingresos totales al DataFrame de comparación
comparison['Ingreso Total Actual'] = ingreso_total_actual
comparison['Ingreso Total Óptimo'] = ingreso_total_optimo

# Mostrar comparación con los ingresos totales
comparison



C:\Users\wmate\AppData\Local\Temp\ipykernel_42388\904039752.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product['log_price'] = np.log(df_product['priceEach'])
C:\Users\wmate\AppData\Local\Temp\ipykernel_42388\904039752.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product['log_quantity'] = np.log(df_product['quantityOrdered'])
C:\Users\wmate\AppData\Local\Temp\ipykernel_42388\904039752.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,Precio Actual,Precio Óptimo,Elasticidad,Tipo de Demanda,Variación Porcentual,Ingreso Total Actual,Ingreso Total Óptimo
Classic Cars,108.00,118.80,0.02,Elástica positiva (productos de lujo),10.00,9604190.61,13167525.42
Motorcycles,87.32,96.06,0.02,Elástica positiva (productos de lujo),10.00,9604190.61,13167525.42
Planes,80.33,88.36,0.01,Elástica positiva (productos de lujo),10.00,9604190.61,13167525.42
Ships,77.75,85.53,0.00,Elástica positiva (productos de lujo),10.00,9604190.61,13167525.42
Trains,67.14,1006.23,-0.07,Inelástica (Demanda inelástica),1398.70,9604190.61,13167525.42
Trucks and Buses,92.71,101.98,0.04,Elástica positiva (productos de lujo),10.00,9604190.61,13167525.42
Vintage Cars,78.36,86.19,0.00,Elástica positiva (productos de lujo),10.00,9604190.61,13167525.42


In [7]:
comparison.sum()

Precio Actual                                                      591.61
Precio Óptimo                                                     1583.15
Elasticidad                                                          0.03
Tipo de Demanda         Elástica positiva (productos de lujo)Elástica ...
Variación Porcentual                                              1458.70
Ingreso Actual                                                   20879.52
Ingreso Óptimo                                                   55362.97
dtype: object